In [7]:
import pandas as pd
from pprint import pprint

In [8]:
file = pd.ExcelFile("TC-guide.xlsx")
file = pd.read_excel(file, "Data")

In [9]:
%%capture
# Fill all the NaNs with 0s.
file = file.fillna(0)
!pip install ipython_blocking

In [10]:
import pickle as pkl
f = open("graph_keys.pkl","rb")
mat = pkl.load(f)
keyfloor = pkl.load(f)
f.close()

In [11]:
start = '2Normal'
distance = {}
pred = {}
keys = {}
# Initialize Bellman Ford distances.
for x in range(101):
  for y in file.columns:
    distance[str(x)+y] = 0 if str(x)+y == start else float('inf')
    keys[str(x)+y] = 0

# Bellman Ford on graph we made above.
for i in range(len(mat)):
  for v in mat:
    for n in mat[v]:
      if "-" in n:
        continue
      distance[n] = min(distance[n], distance[v] + mat[v][n])
      distance[n] = abs(distance[n])
      if distance[n] >= distance[v]+mat[v][n]:
        pred[n] = v 

In [12]:
import ipywidgets as widgets
from IPython.display import display
import ipython_blocking

mytype = widgets.Dropdown(
    options=file.columns,
    value='Normal',
    description='Exit Type:',
    disabled=False,
)
mynum = widgets.BoundedIntText(
    value=2,
    min=2,
    max=99,
    step=1,
    description='Floor No:',
    disabled=False
)
box = widgets.HBox([mytype, mynum])
display(box)

from IPython.display import Javascript, display
from ipywidgets import widgets



button = widgets.Button(description="Find path from 2nd Normal Floor.")
button.style.button_color = 'lightblue'

display(button)

Button(description='Find path from 2nd Normal Floor.', style=ButtonStyle(button_color='lightblue'))

In [ ]:
%blockrun button

In [ ]:
box.close()
button.close()
# Retrieve path from our algorithm above.
it = str(mynum.value) + mytype.value
numk = 0
nums = 0
path = []
out = widgets.Output(layout={'border': '1px solid black'})
while it != start:
    path.append(it)
    it = pred[it]
path.append(it)
prev = None
prevcol = None
i = 1
with out:
    print("Distance: ", distance[path[1]]/99)
    print("Steps to follow:")
    spaces = " "
    for idx,x in enumerate(reversed(path)):
        if i < 10:
            spaces = " "
        else:
            spaces = ""
        if x in keyfloor:
            print(str(i)+".",x, " (Key)")
            numk+=1
            i+=1
        for y in file.columns:
            if y in x:
                mycol = y
                break
        if prevcol != None and prevcol != mycol: 
            print(str(i)+".",prev, " (Switch)")
            nums+=1
            i+=1
        if idx == len(path)-1:
            print(str(i)+".", x, "(Exit)", "\nNeeded: % 2d keys and % 2d switches" % (numk,nums))
        prevcol = mycol
        prev = x
display(out)
def run_all(ev):
    display(Javascript('IPython.notebook.execute_all_cells()'))
    button.close()
    out.close()

button = widgets.Button(description="Go back.")
button.on_click(run_all)
display(button)